In [1]:
import coiled
from dask.distributed import wait, LocalCluster
import dask.dataframe as dd 
import dask

### With Arrow strings

In [2]:
dask.config.set({"dataframe.convert-string": True})

In [3]:
%%time

cluster = LocalCluster()

CPU times: user 259 ms, sys: 62.8 ms, total: 322 ms
Wall time: 760 ms


In [4]:
def read_df():
    parquet_paths = [
        f"s3://coiled-datasets/uber-lyft-tlc/part.{i}.parquet"
        for i in range(10)
    ]
    return dd.read_parquet(parquet_paths).persist()

In [5]:
%%time

client = cluster.get_client()
client

CPU times: user 10.4 ms, sys: 5.19 ms, total: 15.6 ms
Wall time: 13.9 ms


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 12,Total memory: 64.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:56273,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 12
Started: Just now,Total memory: 64.00 GiB
Comm: tcp://127.0.0.1:56285,Total threads: 3
Dashboard: http://127.0.0.1:56290/status,Memory: 16.00 GiB
Nanny: tcp://127.0.0.1:56276,


In [6]:
%%time

df = read_df()

wait(df);

CPU times: user 3.84 s, sys: 1.06 s, total: 4.91 s
Wall time: 40.3 s


### With object strings

In [6]:
%%time

client.restart()

2023-04-14 10:23:14,805 - distributed.nanny - WARNING - Restarting worker
2023-04-14 10:23:14,825 - distributed.nanny - WARNING - Restarting worker
2023-04-14 10:23:14,830 - distributed.nanny - WARNING - Restarting worker
2023-04-14 10:23:14,834 - distributed.nanny - WARNING - Restarting worker


CPU times: user 46.2 ms, sys: 37.7 ms, total: 83.8 ms
Wall time: 535 ms


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 12,Total memory: 64.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:56273,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 12
Started: Just now,Total memory: 64.00 GiB
Comm: tcp://127.0.0.1:56311,Total threads: 3
Dashboard: http://127.0.0.1:56312/status,Memory: 16.00 GiB
Nanny: tcp://127.0.0.1:56276,


In [7]:
dask.config.set({"dataframe.convert-string": False})

In [8]:
%%time

df = read_df()

wait(df);

CPU times: user 4.22 s, sys: 1.22 s, total: 5.44 s
Wall time: 45.4 s


In [9]:
client.close()

In [10]:
cluster.close()